In [58]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#look for educationla attaiment SCHL

In [59]:
#Loading  Florida Data csv file
file_to_load = 'data_files/psam_p12.csv'
#Converting to Pandas df
degree_data = pd.read_csv(file_to_load)
degree_data.head()

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
0,P,2013000000001,5,1,1102,3,12,1061971,16,54,...,23,5,17,29,23,16,20,15,15,16
1,P,2013000000001,5,2,1102,3,12,1061971,18,54,...,36,5,20,33,29,15,19,15,24,21
2,P,2013000000001,5,3,1102,3,12,1061971,33,38,...,47,12,33,56,58,30,36,31,32,38
3,P,2013000000001,5,4,1102,3,12,1061971,31,28,...,50,11,28,49,46,29,29,29,37,30
4,P,2013000000001,5,5,1102,3,12,1061971,20,13,...,35,8,19,39,40,16,19,19,22,17


In [60]:
# getting the degree codes and names (I converted the information to a csv from the pdf)

degree_code = pd.read_csv("data_files/degree_codes.csv", header = None)
degree_code = pd.DataFrame(degree_code)
degree_code = degree_code.rename(columns = {0: "degree_code",
                                            1: "degree_name"})
degree_code.head()

,degree_code,degree_name
0,1100.0,General Agriculture
1,1101.0,Agriculture Production And Management
2,1102.0,Agricultural Economics
3,1103.0,Animal Sciences
4,1104.0,Food Science


In [68]:
#grouping by degree code
obs = degree_data.groupby('FOD1P')
#getting counts of people for each degree
job_counts = pd.DataFrame(obs.count()[['RT']])
#getting mean wage for each degree
income = pd.DataFrame(obs.mean()['WAGP'])

#merging counts and income frames
income_per_degree = job_counts.merge(income, on = 'FOD1P')\
                    .reset_index()
#renaming columns
income_per_degree = income_per_degree.rename(columns = {"FOD1P": "degree_code",
                                                        "RT": "number_of_people",
                                                        "WAGP":"Wage/Salary"})
income_per_degree.head()

,degree_code,number_of_people,Wage/Salary
0,1100.0,466,37466.309013
1,1101.0,226,43958.495575
2,1102.0,92,60917.391304
3,1103.0,352,45752.272727
4,1104.0,204,54200.000000


In [70]:
#grouping data by degree code only including people that are looking for a job
obs_looking = degree_data[degree_data['NWLK'] == 1].groupby('FOD1P')
#getting counts of people for each degree and renaming column
looking_counts = pd.DataFrame(obs_looking.count()[['RT']]).reset_index()
looking_counts = looking_counts.rename(columns = {"FOD1P": "degree_code",
                                                  "RT": "looking"})

looking_counts.head()

,degree_code,looking
0,1100.0,10
1,1101.0,6
2,1102.0,1
3,1103.0,12
4,1104.0,10


In [71]:
low_end_jobs = [4020,4030,4040,4050,4060,4110,4120,4130,4140,4150,4220,4230,4240,4250,4720,4740,4750,4760,4940]
data_low_end = degree_data[degree_data["OCCP"].isin(low_end_jobs)]

low_end_counts = pd.DataFrame(data_low_end.groupby('FOD1P').count()['RT']).reset_index()
low_end_counts = low_end_counts.rename(columns = {"RT" : "Non-Degree Jobs",
                                                  "FOD1P": "degree_code",})

low_end_counts.head()

,degree_code,Non-Degree Jobs
0,1100.0,24
1,1101.0,16
2,1102.0,5
3,1103.0,19
4,1104.0,7


In [98]:
non_college = list(range(12,18))

data_non_college = degree_data[degree_data["SCHL"].isin(non_college)]

non_college_counts = pd.DataFrame(data_non_college.groupby('OCCP').count())[["RT"]].reset_index()
non_college_wage = pd.DataFrame(data_non_college.groupby('OCCP').mean())[["WAGP"]].reset_index()
non_college_data = non_college_counts.merge(non_college_wage, on = 'OCCP')
non_college_data = non_college_data.rename(columns = {"RT" : "Non-College count",
                                                      "OCCP": "Occupation",
                                                      "WAGP":"Wage/Salary"})

non_college_data.head()

,Occupation,Non-College count,Wage/Salary
0,10.0,788,83615.736041
1,20.0,678,56697.050147
2,40.0,13,78538.461538
3,50.0,374,49620.053476
4,60.0,10,69520.000000


In [72]:
#merge to a final dataframe including all data.
full_data = degree_code.merge(income_per_degree, on = 'degree_code')
full_data = full_data.merge(low_end_counts, on = 'degree_code')
full_data = full_data.merge(looking_counts, on = "degree_code")
full_data.head()

,degree_code,degree_name,number_of_people,Wage/Salary,Non-Degree Jobs,looking
0,1100.0,General Agriculture,466,37466.309013,24,10
1,1101.0,Agriculture Production And Management,226,43958.495575,16,6
2,1102.0,Agricultural Economics,92,60917.391304,5,1
3,1103.0,Animal Sciences,352,45752.272727,19,12
4,1104.0,Food Science,204,54200.000000,7,10


In [74]:
#finding percentages
full_data["looking_%"] = full_data["looking"]\
                                /full_data["number_of_people"]\
                                * 100

full_data["NDJ_%"] = full_data["Non-Degree Jobs"]\
                        /full_data["number_of_people"]\
                        * 100
full_data.head()

,degree_code,degree_name,number_of_people,Wage/Salary,Non-Degree Jobs,looking,looking_%,NDJ_%
0,1100.0,General Agriculture,466,37466.309013,24,10,2.145923,5.150215
1,1101.0,Agriculture Production And Management,226,43958.495575,16,6,2.654867,7.079646
2,1102.0,Agricultural Economics,92,60917.391304,5,1,1.086957,5.434783
3,1103.0,Animal Sciences,352,45752.272727,19,12,3.409091,5.397727
4,1104.0,Food Science,204,54200.000000,7,10,4.901961,3.431373


In [84]:
#sorting by wage in descending order
full_data.sort_values('Wage/Salary', ascending = True).head(5)

,degree_code,degree_name,number_of_people,Wage/Salary,Non-Degree Jobs,looking,looking_%,NDJ_%
31,2309.0,Secondary Teacher Education,1486,18490.289367,35,21,1.413190,2.355316
37,2399.0,Miscellaneous Education,1297,18649.629915,26,28,2.158828,2.004626
34,2312.0,Teacher Education: Multiple Levels,595,21158.151261,20,10,1.680672,3.361345
26,2304.0,Elementary Education,10394,21369.873004,199,140,1.346931,1.914566
35,2313.0,Language And Drama Education,1447,21408.313753,31,25,1.727713,2.142364


In [76]:
full_data.sort_values('NDJ_%', ascending = False).head(10)

,degree_code,degree_name,number_of_people,Wage/Salary,Non-Degree Jobs,looking,looking_%,NDJ_%
23,2201.0,Cosmetology Services And Culinary Arts,208,37029.567308,42,12,5.769231,20.192308
105,6099.0,Miscellaneous Fine Arts,38,29089.473684,6,2,5.263158,15.789474
95,5701.0,"Electrical, Mechanical, And Precision Technolo...",53,33941.132075,7,1,1.886792,13.207547
100,6003.0,Visual And Performing Arts,200,36792.000000,24,19,9.500000,12.000000
126,6211.0,Hospitality Management,1680,45999.761905,163,58,3.452381,9.702381
102,6005.0,Film Video And Photographic Arts,344,41972.674419,26,24,6.976744,7.558140
98,6001.0,Drama And Theater Arts,650,34753.892308,47,31,4.769231,7.230769
89,5504.0,Geography,506,37823.932806,36,19,3.754941,7.114625
1,1101.0,Agriculture Production And Management,226,43958.495575,16,6,2.654867,7.079646
64,4101.0,Physical Fitness Parks Recreation And Leisure,1554,42480.868726,106,55,3.539254,6.821107


In [77]:
full_data.sort_values('Non-Degree Jobs', ascending = False).head(10)

,degree_code,degree_name,number_of_people,Wage/Salary,Non-Degree Jobs,looking,looking_%,NDJ_%
119,6203.0,Business Management And Administration,16803,47080.539427,750,622,3.701720,4.463489
117,6200.0,General Business,10947,49884.876222,494,304,2.777017,4.512652
75,5200.0,Psychology,10005,38215.942029,427,358,3.578211,4.267866
24,2300.0,General Education,11095,21540.980622,345,182,1.640379,3.109509
122,6206.0,Marketing And Marketing Research,5408,52457.712648,340,193,3.568787,6.286982
118,6201.0,Accounting,9616,54245.939476,337,295,3.067804,3.504576
12,1901.0,Communications,4131,45458.971193,236,173,4.187848,5.712902
81,5301.0,Criminal Justice And Fire Protection,4134,42442.327044,201,207,5.007257,4.862119
26,2304.0,Elementary Education,10394,21369.873004,199,140,1.346931,1.914566
86,5501.0,Economics,4485,58361.427871,165,111,2.474916,3.678930
